# Simple simpletransformers based classification model for NLP Cup Event #3

In [1]:
# For Google Colab (always needed) or Jupyter Notebook (if you don't have it already)
!pip install simpletransformers

     |████████████████████████████████| 194kB 2.9MB/s 
     |████████████████████████████████| 204kB 8.7MB/s 
     |████████████████████████████████| 3.8MB 7.4MB/s 
     |████████████████████████████████| 675kB 33.7MB/s 
     |████████████████████████████████| 1.1MB 39.9MB/s 
     |████████████████████████████████| 890kB 53.4MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=c2fc9828dbbf50732028668d067356804249630751562f9892b2aabb617bfbf8
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=3f4d5f6e23c0fdae88fc83344ef4429a9e8ab1dcc44bb88cb6cd9db84c98f70b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built seqeval sacremoses


In [0]:
# Import all needed libraries
import numpy as np
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from simpletransformers.classification import ClassificationModel

In [3]:
# Download NLTK stop words
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
# Mount Google Drive so files accessible from Colab notebook - not needed using Jupyter Notebook
from google.colab import drive 
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Read in data - CSV files placed in same folder location as Colab Notebooks
train_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/spamraw_train.csv")

In [0]:
# Pre-processing
def remove_stopwords(text):
  text_to_lst = text.split(' ')
  filtered_words = [word for word in text_to_lst if word not in nltk.corpus.stopwords.words('english')]
  return ' '.join([word for word in filtered_words])

train_df['sms_text_processed'] = train_df['sms_text'].apply(lambda x: remove_stopwords(x))

In [0]:
# Train test split
X_train, X_val, y_train, y_val = train_test_split(train_df['sms_text_processed'], train_df['spam'], test_size=0.10, random_state=42)

train_dataset =  pd.concat([X_train, y_train], axis=1) 
val_dataset = pd.concat([X_val, y_val], axis=1)

In [8]:
# Load a pre-trained model, and train it with our data | See all models available: https://huggingface.co/transformers/pretrained_models.html

# Create model ... args = parameters
args={'reprocess_input_data': True, 'max_seq_length': 300, 'num_train_epochs': 12, 'fp16': False, 'train_batch_size': 4, 'overwrite_output_dir': True}
my_model = ClassificationModel('roberta', 'distilroberta-base', num_labels=2, use_cuda=True, cuda_device=0, args=args)

# Train the model
my_model.train_model(train_dataset)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:267: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Running loss: 0.669999

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.000787


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


...

Running loss: 0.000239


Running loss: 0.000011


Running loss: 0.000005


Running loss: 0.000005


Running loss: 0.000003



In [0]:
# Save model - run only if you want to save the model for later use
import torch
torch.save(my_model, open('/content/drive/My Drive/Colab Notebooks/roberta_model2-torch.pickle', 'wb'))

In [0]:
# Load model - run only if you want to load a saved model - don't run the training step!
import torch
my_model = torch.load(open('/content/drive/My Drive/Colab Notebooks/roberta_model2-torch.pickle', 'rb'))

In [9]:
# Evaluate the model
result, model_outputs, wrong_predictions = my_model.eval_model(val_dataset, acc=f1_score)
pred_val = np.argmax(model_outputs, axis=1).tolist()

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:690: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


In [10]:
print("Results on evaluation:")
print("----------------------")
print("F1 Score = {:.6f}\n".format(f1_score(y_val, pred_val, average='micro') * 100))

print(classification_report(y_val, pred_val))
print(confusion_matrix(y_val, pred_val))

Results on evaluation:
----------------------
F1 Score = 99.600000

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       441
           1       1.00      0.97      0.98        59

    accuracy                           1.00       500
   macro avg       1.00      0.98      0.99       500
weighted avg       1.00      1.00      1.00       500

[[441   0]
 [  2  57]]


In [20]:
# Show 2 errors 
val_dataset[y_val != pred_val]

,sms_text_processed,spam
1073,Would like see XXX pics hot nearly banned uk!,1
315,"Hello darling today? I would love chat, dont t...",1


## And now for the true test ...

In [0]:
# Load the test set, and solution set to compare scores
test_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/spamraw_test.csv")
sol_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/spamraw_test_solutions.csv")
y_test = sol_df['spam']

In [0]:
# Pre-processing, as always, exact same as with train set
test_df['sms_text_processed'] = test_df['sms_text'].apply(lambda x: remove_stopwords(x))

In [15]:
# Get predictions using our trained model
pred_test, _ = my_model.predict(test_df['sms_text_processed'])

In [16]:
print("Results on test set:")
print("--------------------")
print("F1 Score = {:.6f}\n".format(f1_score(y_test, pred_test, average='micro') * 100))
print(classification_report(y_test, pred_test))
print(confusion_matrix(y_test, pred_test))

Results on test set:
--------------------
F1 Score = 99.463327

              precision    recall  f1-score   support

           0       0.99      1.00      1.00       485
           1       1.00      0.96      0.98        74

    accuracy                           0.99       559
   macro avg       1.00      0.98      0.99       559
weighted avg       0.99      0.99      0.99       559

[[485   0]
 [  3  71]]


In [21]:
# Show 3 errors - incorrectly predicted as not spam
test_df[y_test != pred_test]

,id,sms_text,sms_text_processed
102,12102,RCT' THNQ Adrian for U text. Rgds Vatian,RCT' THNQ Adrian U text. Rgds Vatian
136,12136,This message is brought to you by GMW Ltd. and...,This message brought GMW Ltd. connected
512,12512,dating:i have had two of these. Only started a...,dating:i two these. Only started sent text tal...
